# OpenAI API 사용 Function Calling 
: https://www.promptingguide.ai/kr/applications/function_calling

In [15]:
from dotenv import load_dotenv
import os
import json

# .env 파일 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

In [3]:
# 함수 정의
def get_current_weather(location):
    # 이 함수는 날씨 정보를 반환한다고 가정합니다.
    print('get_current_weather function is called!!')    
    return {
        "location": location,
        "temperature": "25°C",
        "description": "구름 조금 있고 바람 붐"
    }

# Function Calling을 위한 함수 메타데이터 정의
functions = [
    {
        "name": "get_current_weather",
        "description": "지정된 위치의 현재 날씨를 가져옵니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "도시와 국가 예: 서울, 대한민국"
                }
            },
            "required": ["location"]
        }
    }
]

In [19]:
# 사용자가 원하는 위치의 날씨를 묻는 프롬프트
prompt = '서울의 날씨는 어떤가요?'
# prompt = '뉴욕의 날씨는 어떤가요?'
# prompt = '파리의 날씨는 어떤가요?'
# prompt = '오늘 주식 시세는 어떤가요?'

# OpenAI API 호출
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[ {"role": "user", "content": prompt}],
    functions=functions,
    function_call="auto"  # Function Calling을 활성화하기 위한 설정
)

# print(response.choices[0].message.content)   # None
print(response)
print('-'*100)

# Function Call이 발생했는지 확인 후 직접 함수 호출
if response.choices[0].finish_reason == 'function_call':
    function_call = response.choices[0].message.function_call

    if function_call.name == 'get_current_weather':
        location = json.loads(function_call.arguments)['location']
        weather_info = get_current_weather(location)
        
        # 사용자가 요청한 정보에 대한 응답 생성
        print(f"{location}의 현재 날씨는 {weather_info['temperature']}이고, {weather_info['description']}입니다.") 

else:
    # Function Calling이 발생하지 않았을 때의 응답 처리
    print(response.choices[0].message.content)  

ChatCompletion(id='chatcmpl-CMC6mUokYfutzFeT09cuAfYc7bxE9', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"뉴욕, 미국"}', name='get_current_weather'), tool_calls=None))], created=1759405796, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=18, prompt_tokens=70, total_tokens=88, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
----------------------------------------------------------------------------------------------------
get_current_weather function is called!!
뉴욕, 미국의 현재 날씨는 25°C이고, 구름 조금 있고 바람 붐입니다.


In [8]:
# function_call 사용하지 않는 질의
prompt = '서울의 날씨는 어떤가요?'

# OpenAI API 호출
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[ {"role": "user", "content": prompt}],
    # functions=functions,
    # function_call="auto" 
)

print(response.choices[0].message.content)  
print(response)  # function_call=None

죄송하지만, 실시간 정보나 현재의 날씨를 제공할 수는 없습니다. 서울의 최신 날씨 정보를 확인하려면 기상청 웹사이트나 날씨 앱을 이용해 보세요. 보통 서울의 날씨는 봄에는 온화하고, 여름에는 덥고 습하며, 가을에는 선선하고 겨울에는 춥습니다. 특정한 날짜나 기간에 대한 예보가 필요하시다면, 해당 정보를 찾아보시는 것이 좋습니다!
ChatCompletion(id='chatcmpl-CMBpNSVUpsoGJSXZ3qWaJg0bmrI0l', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='죄송하지만, 실시간 정보나 현재의 날씨를 제공할 수는 없습니다. 서울의 최신 날씨 정보를 확인하려면 기상청 웹사이트나 날씨 앱을 이용해 보세요. 보통 서울의 날씨는 봄에는 온화하고, 여름에는 덥고 습하며, 가을에는 선선하고 겨울에는 춥습니다. 특정한 날짜나 기간에 대한 예보가 필요하시다면, 해당 정보를 찾아보시는 것이 좋습니다!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1759404717, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=103, prompt_tokens=16, total_tokens=119, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), p

### Dummy Function 정의

In [21]:
# 현재 날씨를 가져오는 dummy 함수를 정의
# 실제 검색 가능한 라이브러리 함수와 연동하여 사용한다
def get_current_weather(location,unit='섭씨'):
    # 이 함수는 날씨 정보를 반환한다고 가정합니다.
    print('get_current_weather function is called!!')
    weather = {
        "location": location,
        "temperature": "30°C",
        "unit": unit,
    }
    return json.dumps(weather)    # dict --> json str

### 함수 메타데이터 정의

OpenAI 문서에서 시연된 바와 같이, 요청에 포함될 함수를 정의하는 간단한 예시가 있습니다.<br>
descriptions는 매우 중요합니다. 이 설명은 LLM에 직접 전달되며, LLM은 descriptions을 바탕으로 함수를 사용할지, 그리고 어떻게 호출할지를 결정하게 됩니다.

In [24]:
# define a function as tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "주어진 위치의 현재 날씨를 가져오기",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "도시, 예. 서울",
                    },
                    "unit": {
                        "type": "string", 
                        "enum": ["섭씨", "화씨"]},
                },
                "required": ["location"],
            },
        },   
    }
]

In [23]:
# define a list of messages
messages = [
    {
        'role' : 'user',
        'content' : '서울의 날씨는 어떤가요?'
    }
]

In [25]:
# 질의 함수 정의하기
def get_completion(messages,model="gpt-4o-mini",temperature=0,max_tokens=300,tools=None,tool_choice=None):
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature,
        max_tokens = max_tokens,
        tools = tools,
        tool_choice = tool_choice
    )
    return response.choices[0].message 

In [26]:
response = get_completion(messages,tools=tools)  # function_call 발생
print(response)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_jWAQumIAAgiBsXCGYCzdshdG', function=Function(arguments='{"location":"서울","unit":"섭씨"}', name='get_current_weather'), type='function')])


In [27]:
response.tool_calls[0].function.arguments

'{"location":"서울","unit":"섭씨"}'

In [28]:
# JSON String --> dict
args = json.loads(response.tool_calls[0].function.arguments)
print(args)   # dict

{'location': '서울', 'unit': '섭씨'}


In [33]:
get_current_weather(**args)   # 키워드 인수
# get_current_weather(location = '서울', unit = '섭씨') 과 동일한 결과

get_current_weather function is called!!
get_current_weather function is called!!


'{"location": "\\uc11c\\uc6b8", "temperature": "30\\u00b0C", "unit": "\\uc12d\\uc528"}'

### Function Calling 동작 제어하기
LLM 기반 대화 에이전트의 맥락에서 이 function_calling 기능을 설계하는 데 관심이 있다고 가정해 보겠습니다. 그런 다음 솔루션은 어떤 함수를 호출해야 하는지 또는 호출해야 하는지 알아야 합니다. 인사말 메시지의 간단한 예를 살펴보겠습니다.

In [44]:
messages = [
    {
        'role' : 'user',
        # 'content' : '안녕 반가워?'
        'content' : '서울 날씨 알려줘?'
    }
]

In [45]:
get_completion(messages,tools=tools)  # 'auto'로 동작 

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_iiYrStCtIPgVHAeKl6awUDGT', function=Function(arguments='{"location":"서울","unit":"섭씨"}', name='get_current_weather'), type='function')])

- 함수 호출의 동작을 제어하기 위해 원하는 동작을 지정할 수 있습니다. 기본적으로, 모델은 함수를 호출할지 여부와 어떤 함수를 호출할지를 스스로 결정합니다. 이는 기본 설정인 tool_choice: "auto"를 설정함으로써 이루어집니다.

In [46]:
get_completion(messages,tools=tools,tool_choice='auto')  # 'auto'로 동작 

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_f9OnHekcDixmc3jseyiQlb3x', function=Function(arguments='{"location":"서울","unit":"섭씨"}', name='get_current_weather'), type='function')])

- tool_choice: "none"으로 설정하면 모델이 제공된 함수 중 어떤 것도 사용하지 않도록 강제합니다.

In [47]:
get_completion(messages,tools=tools,tool_choice='none') # tool_calls=None

ChatCompletionMessage(content='섭씨로 서울의 현재 날씨를 확인해볼게요. 잠시만 기다려 주세요.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

In [48]:
get_completion(messages,tools=tools,tool_choice=None)  # 'auto'로 동작 

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_lFMuSaMUXKfiOCcu26DSvISN', function=Function(arguments='{"location":"서울","unit":"섭씨"}', name='get_current_weather'), type='function')])

In [40]:
print(type(None))   # None과 'none'은 서로 다르다

<class 'NoneType'>


- 또한, 애플리케이션에서 원하는 동작이 있다면 모델이 반드시 함수를 선택하도록 강제할 수도 있습니다.
예시:

In [54]:
messages = [
    {
        'role' : 'user',
        'content' : '부산의 날씨는 어때?'
    }
]
get_completion(messages,tools=tools,tool_choice={"type": "function", "function": {"name": "get_current_weather"}})

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_UeBsZNDXcci5EuONNW6i5GlC', function=Function(arguments='{"location":"부산","unit":"섭씨"}', name='get_current_weather'), type='function')])

In [55]:
messages = [
    {
        'role' : 'user',
        'content' : '안녕 반가워?'
    }
]
get_completion(messages,tools=tools,tool_choice={"type": "function", "function": {"name": "get_current_weather"}})
# 인자값을 얻어 올수 없어서 함수 호출 시 인자 전달 오류가 발생

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_0qQguT4gvTEVOVT2euLBn4C8', function=Function(arguments='{"location":"서울"}', name='get_current_weather'), type='function')])

- OpenAI API는 하나의 턴에서 여러 함수를 호출할 수 있는 병렬 함수 호출도 지원합니다. 

In [56]:
# 질문안에 2개의 location이 들어 있는 경우
messages = [
    {
        'role' : 'user',
        'content' : '다음 며칠 동안 서울과 부산의 날씨는 어떤가요?'
    }
]
get_completion(messages,tools=tools)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_fFDiZmfHeSw2rWqIgSpKXxri', function=Function(arguments='{"location": "서울", "unit": "섭씨"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_BGJebSkjoZQsXeopteKk1qlX', function=Function(arguments='{"location": "부산", "unit": "섭씨"}', name='get_current_weather'), type='function')])

### 모델 피드백을 위한 Function Calling Response 

또한, 함수 호출로 생성된 입력을 사용하여 API를 호출한 후 얻은 결과를 반환하는 에이전트를 개발하는 것에 관심이 있을 수 있습니다. 다음은 그 예시입니다:

In [69]:
messages = []
messages.append({"role": "user", "content": "서울의 날씨는 어때!"})
assistant_message = get_completion(messages,tools=tools,tool_choice='auto')
assistant_message = json.loads(assistant_message.model_dump_json())
assistant_message["content"] = str(assistant_message['tool_calls'][0]['function'])
print(assistant_message)

del assistant_message['function_call']

{'content': '{\'arguments\': \'{"location":"서울","unit":"섭씨"}\', \'name\': \'get_current_weather\'}', 'refusal': None, 'role': 'assistant', 'annotations': [], 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_eBPTcLUuILpXXwOhkRidWf0u', 'function': {'arguments': '{"location":"서울","unit":"섭씨"}', 'name': 'get_current_weather'}, 'type': 'function'}]}


In [71]:
messages.append(assistant_message)
messages

[{'role': 'user', 'content': '서울의 날씨는 어때!'},
 {'content': '{\'arguments\': \'{"location":"서울","unit":"섭씨"}\', \'name\': \'get_current_weather\'}',
  'refusal': None,
  'role': 'assistant',
  'annotations': [],
  'audio': None,
  'tool_calls': [{'id': 'call_eBPTcLUuILpXXwOhkRidWf0u',
    'function': {'arguments': '{"location":"서울","unit":"섭씨"}',
     'name': 'get_current_weather'},
    'type': 'function'}]}]

그런 다음 get_current_weather 함수의 결과를 추가하고 이를 도구 역할을 사용하여 모델에 다시 전달합니다

In [72]:
# get the weather information to pass back to the model
weather = get_current_weather(messages[1]["tool_calls"][0]["function"]["arguments"])

messages.append({"role": "tool",
                 "tool_call_id": assistant_message["tool_calls"][0]["id"],
                 "name": assistant_message["tool_calls"][0]["function"]["name"],
                 "content": weather})

get_current_weather function is called!!


In [73]:
messages

[{'role': 'user', 'content': '서울의 날씨는 어때!'},
 {'content': '{\'arguments\': \'{"location":"서울","unit":"섭씨"}\', \'name\': \'get_current_weather\'}',
  'refusal': None,
  'role': 'assistant',
  'annotations': [],
  'audio': None,
  'tool_calls': [{'id': 'call_eBPTcLUuILpXXwOhkRidWf0u',
    'function': {'arguments': '{"location":"서울","unit":"섭씨"}',
     'name': 'get_current_weather'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_eBPTcLUuILpXXwOhkRidWf0u',
  'name': 'get_current_weather',
  'content': '{"location": "{\\"location\\":\\"\\uc11c\\uc6b8\\",\\"unit\\":\\"\\uc12d\\uc528\\"}", "temperature": "30\\u00b0C", "unit": "\\uc12d\\uc528"}'}]

In [74]:
final_response = get_completion(messages,tools=tools)

In [75]:
final_response

ChatCompletionMessage(content='서울의 현재 기온은 30도 섭씨입니다. 날씨가 덥네요!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

In [77]:
# Function 정의 (OpenAI에게 전달할 스펙)
tools = [
    {
        "type": "function",
        "function": {
            "name": "add_numbers",
            "description": "두 숫자를 더한다",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "첫 번째 숫자"},
                    "b": {"type": "number", "description": "두 번째 숫자"}
                },
                "required": ["a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_text",
            "description": "주어진 텍스트를 분석한다",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {"type": "string", "description": "분석할 텍스트"}
                },
                "required": ["text"]
            }
        }
    }
]

# 실제 함수 구현
def add_numbers(a, b):
    return {"result": a + b}

def analyze_text(text):
    import collections
    words = text.split()
    word_count = len(words)
    char_count = len(text)
    most_common = collections.Counter(words).most_common(1)[0][0] if words else None
    return {
        "word_count": word_count,
        "char_count": char_count,
        "most_common_word": most_common
    }

In [79]:
import json

# 사용자 입력
messages = [
    {"role": "user", "content": "345와 789를 더해줘"},
    {"role": "user", "content": "이 문장의 단어 수와 가장 많이 나온 단어를 분석해줘: hello world hello GPT"}
]

# 각 메시지마다 처리
for user_input in messages:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[user_input],
        tools=tools,
        tool_choice="auto"
    )

    msg = response.choices[0].message
    print(msg)
    if msg.tool_calls:
        for tool_call in msg.tool_calls:
            function_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)

            if function_name == "add_numbers":
                function_response = add_numbers(**arguments)
            elif function_name == "analyze_text":
                function_response = analyze_text(**arguments)

            # 함수 결과 전달 후 GPT 응답 생성
            follow_up = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    user_input,
                    {
                        "role": "assistant",
                        "tool_calls": [tool_call]
                    },
                    {
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "name": function_name,
                        "content": json.dumps(function_response)
                    }
                ]
            )
            print('>>응답:',follow_up.choices[0].message.content)
            
    else:
        print(msg.content)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_K3pvMRJG3daBfxl7TWfkMlG8', function=Function(arguments='{"a":345,"b":789}', name='add_numbers'), type='function')])
>>응답: 345와 789를 더한 결과는 1134입니다.
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_fzUphkpzrgrW3HonyjILYSue', function=Function(arguments='{"text": "hello world hello GPT"}', name='analyze_text'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_XdKqQlm5WExMs906L18fNgsF', function=Function(arguments='{"a": 3, "b": 0}', name='add_numbers'), type='function')])
>>응답: 주어진 문장 "hello world hello GPT"의 분석 결과는 다음과 같습니다:

- 단어 수: 4
- 문자 수: 21
- 가장 많이 나온 단어: "hello" (두 번 등장)
>>응답: 주어진 문장 "hello world hello GPT"의 단어 수는 5개입니다. 

가장 많이 나온 단어는 "hello"로, 총 2회 등장합니다.
